In [103]:
# Alican Bayındır
# ELM 472 - ODEV2
# Naive Bayes ile spam mail detection

import pandas as pd
import numpy as np
import re

np.set_printoptions(threshold=sys.maxsize)


In [113]:
df = pd.read_csv('spam_ham_dataset.csv', encoding='ISO-8859-1')
df = df.drop(["Unnamed: 0"], axis="columns")
df.columns = ['labels', 'text','label_nums']
df.head()

,labels,text,label_nums
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [114]:
# datayı test ve training için ayırma
data_randomized = df.sample(frac=1, random_state=1)
training_test_index = round(len(data_randomized) * 0.75)
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(3878, 3)
(1293, 3)


In [115]:
# datada tekrar eden alakasız kelimeleri silme
training_set['text'] = training_set['text'].str.replace('\W', ' ') # Removes punctuation
training_set['text'] = training_set['text'].replace('\r'  , ' ')
training_set['text'] = training_set['text'].replace(';'   , ' ')
training_set['text'] = training_set['text'].replace('.'   , ' ')
training_set['text'] = training_set['text'].replace('\''  , ' ')
training_set['text'] = training_set['text'].replace(':'   , ' ')
training_set['text'] = training_set['text'].replace('\n'  , ' ')
training_set['text'] = training_set['text'].replace('#'   , ' ')
training_set['text'] = training_set['text'].replace("/"   , " ")
training_set['text'] = training_set['text'].replace("-"   , " ")
training_set['text'] = training_set['text'].replace("/"   , " ")
training_set['text'] = training_set['text'].replace("|"   , " ")
training_set['text'] = training_set['text'].replace(","   , " ")
training_set['text'] = training_set['text'].replace(">"   , " ")
training_set['text'] = training_set['text'].replace("|"   , " ")
training_set['text'] = training_set['text'].replace("("   , " ")
training_set['text'] = training_set['text'].replace(")"   , " ")
training_set['text'] = training_set['text'].replace("["   , " ")
training_set['text'] = training_set['text'].replace("]"   , " ")
training_set['text'] = training_set['text'].replace("1"   , " ")
training_set['text'] = training_set['text'].replace("2"   , " ")
training_set['text'] = training_set['text'].replace("3"   , " ")
training_set['text'] = training_set['text'].replace("4"   , " ")
training_set['text'] = training_set['text'].replace("5"   , " ")
training_set['text'] = training_set['text'].replace("6"   , " ")
training_set['text'] = training_set['text'].replace("7"   , " ")
training_set['text'] = training_set['text'].replace("8"   , " ")
training_set['text'] = training_set['text'].replace("9"   , " ")
training_set['text'] = training_set['text'].replace("0"   , " ")
training_set['text'] = training_set['text'].replace("    ", " ")
training_set['text'] = training_set['text'].replace("  "  , " ")
training_set['text'] = training_set['text'].replace("  "  , " ")
training_set['text'] = training_set['text'].replace("+"   , " ")
training_set['text'] = training_set['text'].replace("we ' re"     , "we are")
training_set['text'] = training_set['text'].replace("they ' re","they are")
training_set['text'] = training_set['text'].replace("you ' re","you are")
training_set['text'] = training_set['text'].replace("Subject"," ")
training_set['text'] = training_set['text'].str.lower()
training_set['text'] = training_set['text'].str.split()



C:\Users\Alican\AppData\Local\Temp/ipykernel_1008/3722485601.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set['text'] = training_set['text'].str.replace('\W', ' ') # Removes punctuation


In [ ]:
WholeWords = []
for data in training_set['text']:
   for word in data:
      WholeWords.append(word)

WholeWords = list(set(WholeWords))
TotalNumberOfWords = len(WholeWords)
WordsPerData = {unique_word: [0] * len(training_set['text']) for unique_word in WholeWords}

for i, data in enumerate(training_set['text']):
   for j in data:
      WordsPerData[j][i] += 1

WordCounts = pd.DataFrame(WordsPerData)

training_set_clean = pd.concat([training_set, WordCounts], axis=1)

 Naive Bayes classifier

In [ ]:
# spam olasılıklarının hesabı
SpamMessages = training_set_clean[training_set_clean['labels'] == 'spam']
HamMessages = training_set_clean[training_set_clean['labels'] == 'ham']

ProbabilitySpam = len(SpamMessages) / len(training_set_clean)
ProbabilityHam = len(HamMessages) / len(training_set_clean)

n_words_per_spam_message = SpamMessages['text'].apply(len)
n_spam = n_words_per_spam_message.sum()

n_words_per_ham_message = HamMessages['text'].apply(len)
n_ham = n_words_per_ham_message.sum()

n_vocabulary = len(WholeWords)

parameters_spam = {unique_word:0 for unique_word in WholeWords}
parameters_ham = {unique_word:0 for unique_word in WholeWords}

#her bir kelime için olasılık hesabı
for word in WholeWords:
   n_word_given_spam = SpamMessages[word].sum()
   p_word_given_spam = (n_word_given_spam + 1) / (n_spam + n_vocabulary)
   parameters_spam[word] = p_word_given_spam

   n_word_given_ham = HamMessages[word].sum()
   p_word_given_ham = (n_word_given_ham + 1) / (n_ham + n_vocabulary)
   parameters_ham[word] = p_word_given_ham

In [ ]:
def classify_test_set(message):
   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = ProbabilitySpam
   p_ham_given_message = ProbabilityHam

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham:
         p_ham_given_message *= parameters_ham[word]

   if p_ham_given_message > p_spam_given_message:
      #return 'ham'
        #print("It is a ham mail.")
        return 0
   elif p_spam_given_message > p_ham_given_message:
      #return 'spam'
        #print("It is a spam mail.")
        return 1
   else:
        #print("It is a ham mail.")
        return 0

In [ ]:
test_set['predicted'] = test_set['text'].apply(classify_test_set)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
report = classification_report(test_set['label_nums'],test_set['predicted'])
print(report)

In [109]:
classify_test_set('Our way of saying Thanks for being a part of our community! Building the tools to support and inspire active creators like you is what keeps us going at VSCO.')

0

In [110]:
classify_test_set("Spank monster daddy is on discount only and only just for you!")

1